# Model Analysis

Import statements:

In [1]:
import sys
sys.path.append('/')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from data_processing.batch_generator import load_train_set, load_test_set, load_pre_data, \
                                            load_train_sample, get_class_weights
from machine_learning.models import DoubleStateTrainAsync, DoubleStateTrainSync, DoubleStateProduction, FeedzaiTrainAsync, FeedzaiTrainSync, FeedzaiProduction, BATCH_SIZE
from machine_learning.pipeline import fit_cycle, compile_model
from pandas import DataFrame
import tensorflow as tf

print(os.getcwd())

/home/nancio/Documents/repos/Thesis/src


Set random seed:

In [2]:
tf.random.set_seed(42)  # 42 is a random number for the seed generation

Define models:

In [3]:
train_model = FeedzaiTrainAsync(name="Feedzai_train_async")
production_model = FeedzaiProduction(name="Feedzai_production_async")

Load data:

In [4]:
# For training loop
train_set = load_train_set(batch_size=BATCH_SIZE)
pre_test_data = load_pre_data()
test_set = load_test_set()

# Samples for analysis
train_sample = load_train_sample(300)

Compile model & initialize weights:

In [5]:
compile_model(train_model, train_set)
compile_model(production_model, test_set)

Train model for a few epochs without validating

In [6]:
for epoch in range(5):
        print(f"[EPOCH {epoch}]")

        train_results, val_results = fit_cycle(training_model=train_model, 
                production_model=production_model,
                train_dataset=train_set,
                pre_test_dataset=pre_test_data,
                test_dataset=test_set,
                class_weights=get_class_weights()
                )

[EPOCH 0]
555719/555719 [==============================] - 327s 588us/step - loss: 0.2214 - binary_accuracy: 0.9917 - true_positives_1: 5.0000 - true_negatives_1: 551122.0000 - false_positives_1: 2452.0000 - false_negatives_1: 2140.0000
[EPOCH 1]
555719/555719 [==============================] - 331s 596us/step - loss: 0.2214 - binary_accuracy: 0.9917 - true_positives_1: 5.0000 - true_negatives_1: 551122.0000 - false_positives_1: 2452.0000 - false_negatives_1: 2140.0000
[EPOCH 2]
555719/555719 [==============================] - 332s 598us/step - loss: 0.2214 - binary_accuracy: 0.9917 - true_positives_1: 5.0000 - true_negatives_1: 551122.0000 - false_positives_1: 2452.0000 - false_negatives_1: 2140.0000
[EPOCH 3]
 219/1267 [====>.........................] - ETA: 13:06 - loss: 0.1431 - binary_accuracy: 0.9921 - true_positives: 0.0000e+00 - true_negatives: 222495.0000 - false_positives: 0.0000e+00 - false_negatives: 1761.0000

KeyboardInterrupt: 

In [ ]:
train_model.fit(train_set, 
                epochs=5, 
                class_weight=get_class_weights(), 
                verbose='auto', 
                shuffle=True)

Epoch 1/5
1267/1267 [==============================] - 854s 673ms/step - loss: 0.0529 - binary_accuracy: 0.9941 - true_positives: 0.0000e+00 - true_negatives: 1288863.0000 - false_positives: 143.0000 - false_negatives: 7506.0000
Epoch 2/5
1267/1267 [==============================] - 746s 589ms/step - loss: 0.0515 - binary_accuracy: 0.9942 - true_positives: 0.0000e+00 - true_negatives: 1289006.0000 - false_positives: 0.0000e+00 - false_negatives: 7506.0000
Epoch 3/5
1267/1267 [==============================] - 849s 670ms/step - loss: 0.0511 - binary_accuracy: 0.9942 - true_positives: 0.0000e+00 - true_negatives: 1289006.0000 - false_positives: 0.0000e+00 - false_negatives: 7506.0000
Epoch 4/5
1267/1267 [==============================] - 869s 686ms/step - loss: 0.0508 - binary_accuracy: 0.9942 - true_positives: 0.0000e+00 - true_negatives: 1289006.0000 - false_positives: 0.0000e+00 - false_negatives: 7506.0000
Epoch 5/5
1267/1267 [==============================] - 857s 677ms/step - loss:

In [8]:
train_weight_path = f'machine_learning/saved_models/{train_model.name}'
train_model.save_weights(
       filepath=train_weight_path,
       save_format='h5'
)
#production_model.set_weights(training_model.get_weights())
production_model.load_weights(train_weight_path, by_name=True)
production_model.reset_gru()
sys.stdout.write("\tLoad state: ")
production_model.evaluate(pre_test_data, batch_size=1)

200/200 [==============================] - 0s 543us/step - loss: 1.1737 - binary_accuracy: 0.0850 - true_positives_1: 0.0000e+00 - true_negatives_1: 17.0000 - false_positives_1: 183.0000 - false_negatives_1: 0.0000e+00


[1.1736805438995361, 0.08500000089406967, 0.0, 17.0, 183.0, 0.0]

In [9]:
evaluation_results = production_model.evaluate(test_set, batch_size=1)

555719/555719 [==============================] - 329s 592us/step - loss: 1.2122 - binary_accuracy: 0.0494 - true_positives_1: 2058.0000 - true_negatives_1: 25411.0000 - false_positives_1: 528163.0000 - false_negatives_1: 87.0000
